Poisson Equation on a Unit Square with Dirichlet Boundary Conditions Solver

In [5]:
#--------------------------------------------------------------------#

# All problem inputs must either be a lambda function in x or a real number.

# Input Source Term.
source = lambda x : -6*x[0] -6*x[1]

# Define Dirchlet boundary conditions on the square (includes left, right, bottom and top).
u_L = lambda x: x[1]**3
u_R = lambda x: x[1]**3 + 1

u_B = lambda x: x[0]**3
u_T = lambda x: x[0]**3 + 1

# Input an exact solution to provide error estimates (optional).


#--------------------------------------------------------------------#

In [6]:
from mpi4py import MPI

import matplotlib.pyplot as plt
import numpy as np
import pyvista

import basix.ufl
from dolfinx import fem, mesh, plot, default_scalar_type
import ufl


domain = mesh.create_unit_square(MPI.COMM_WORLD, 64, 64, mesh.CellType.quadrilateral) # Need a circular domain
V = fem.functionspace(domain, ("Lagrange", 1))

In [7]:
dofs_L = fem.locate_dofs_geometrical(V, lambda x: np.isclose(x[0], 0))
dofs_R = fem.locate_dofs_geometrical(V, lambda x: np.isclose(x[0], 1))
dofs_B = fem.locate_dofs_geometrical(V, lambda x: np.isclose(x[1], 0))
dofs_T = fem.locate_dofs_geometrical(V, lambda x: np.isclose(x[1], 1))


boundary_values = [u_L, u_R, u_B, u_T]
dofs = [dofs_L, dofs_R, dofs_B, dofs_T]
bcs = [ _,_,_,_ ]

for i, boundary_value in enumerate(boundary_values):
    if str(boundary_value).isnumeric():
        boundary_value = default_scalar_type(boundary_value)
        bcs[i] = fem.dirichletbc(boundary_value, dofs[i], V)
    else:
        boundary_function = fem.Function(V)
        boundary_function.interpolate(boundary_value)
        bcs[i] = fem.dirichletbc(boundary_function, dofs[i])


In [8]:
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
if callable(source):
    f = fem.Function(V)
    f.interpolate(source)
else:
    f = fem.Constant(domain, default_scalar_type(0))
a = ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = f * v * ufl.dx 

problem = fem.petsc.LinearProblem(a, L, bcs=bcs, petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

AttributeError: module 'dolfinx.fem' has no attribute 'petsc'

In [112]:
if callable(exact_u):
    V2 = fem.functionspace(domain, ("Lagrange", 2))
    uex = fem.Function(V2)
    uex.interpolate(lambda x: x[0]**3+x[1]**3)
else:
    uex = fem.Constant(domain, default_scalar_type(exact_u))

L2_error = fem.form(ufl.inner(uh - uex, uh - uex) * ufl.dx)
error_local = fem.assemble_scalar(L2_error)
error_L2 = numpy.sqrt(domain.comm.allreduce(error_local, op=MPI.SUM))

uex_1 = fem.Function(V)
uex_1.interpolate(uex)
error_max = numpy.max(numpy.abs(uex_1.x.array-uh.x.array)) 
error_max = MPI.COMM_WORLD.allreduce(error_max, op=MPI.MAX)

# Only print the error on one process
if domain.comm.rank == 0:
    print(f"Error_L2 : {error_L2:.10e}")
    print(f"Error_max : {error_max:.10e}")

Error_L2 : 1.3917895658e-04
Error_max : 2.0872192863e-14


In [113]:
pyvista.set_jupyter_backend('html')

domain.topology.create_connectivity(tdim, tdim)
topology, cell_types, geometry = plot.vtk_mesh(domain, tdim)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)
plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()
if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    figure = plotter.screenshot("fundamentals_mesh.png")

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

In [114]:
u_topology, u_cell_types, u_geometry = plot.vtk_mesh(V)
u_grid = pyvista.UnstructuredGrid(u_topology, u_cell_types, u_geometry)
u_grid.point_data["u"] = uh.x.array.real
u_grid.set_active_scalars("u")
u_plotter = pyvista.Plotter()
u_plotter.add_mesh(u_grid, show_edges=False)
u_plotter.view_xy()
if not pyvista.OFF_SCREEN:
    u_plotter.show()

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

In [115]:
warped = u_grid.warp_by_scalar()
plotter2 = pyvista.Plotter()
plotter2.add_mesh(warped, show_edges=True, show_scalar_bar=True)
if not pyvista.OFF_SCREEN:
    plotter2.show()

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…